In [234]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, LSTM, Dropout
from functools import reduce
import re
import json

In [2]:
# GPU testing
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [3]:
df = pd.read_csv('./Documents/scripts/cleaned_corpora.csv')

# removes punctuation for simplicity
df.message = df.message.apply(lambda m: re.sub(r'[:\.\?\-!,]', '', m))

In [4]:
alphabet = df.message.tolist()
alphabet = map(set, alphabet)
alphabet = reduce(lambda s, u: s.union(u), alphabet, set([]))
print(alphabet)

{'z', 'b', 'n', 'q', 'h', 'v', 'a', 's', 'd', '5', 'p', '9', 'j', 'y', 'g', '0', 'k', 'i', 'c', 'x', "'", 'u', '2', '1', 't', ' ', 'l', 'r', '7', '6', 'm', '8', 'w', 'f', '4', 'e', 'o', '3'}


In [5]:
# dictionary = df.message.tolist()
# dictionary = map(lambda m: set(m.split(' ')), dictionary)
# dictionary = reduce(lambda s, u: s.union(u), dictionary, set([]))
# print(len(dictionary))

In [81]:
df_ = df[df.star_sign_index == 1]

X_len = df_.message.shape[0]
X_train_len = int(X_len * 0.95)

X_train = df_.message[:X_train_len]
X_test = df_.message[X_train_len:]

In [82]:
X_train.shape

(3730,)

In [83]:
max_words = 30000
        tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train.values)
sequences = tokenizer.texts_to_sequences(X_train.values)

In [108]:
VOCABULARY_SIZE = len(tokenizer.word_index)

In [109]:
pd.Series(map(len, sequences)).describe()

count    3730.000000
mean       73.820107
std        10.751842
min        46.000000
25%        65.000000
50%        72.000000
75%        83.000000
max       115.000000
dtype: float64

In [110]:
TOKENS_BEFORE_PREDICTION = 20
TOKENS_PREDICTED = 1

In [111]:
def slide_window(xs):
    return np.array([xs[i : i + TOKENS_BEFORE_PREDICTION] for i in range(len(xs) - TOKENS_BEFORE_PREDICTION)])

In [175]:
X_train = np.vstack(pd.Series(sequences).apply(slide_window).values)
X_train = X_train[:40000]

Y_train = X_train[:, (TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED):]
X_train = X_train[:, :(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED)]

def as_dummies(x):
    temp = np.zeros(VOCABULARY_SIZE)
    temp[x - 1] = 1
    return np.array(temp)

Y_train = np.array(list(map(as_dummies, Y_train.reshape(-1))))

In [176]:
print(X_train.shape)
print(Y_train.shape)

(40000, 19)
(40000, 11299)


In [177]:
model_v1 = Sequential([
    Embedding(VOCABULARY_SIZE + 1, 100, input_length=(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED)),
    LSTM(200, return_sequences=True),
    LSTM(200),
    Dense(150, activation='relu'),
    Dropout(0.15),
    Dense(VOCABULARY_SIZE, activation='softmax')
])

In [178]:
model_v1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 19, 100)           1130000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 19, 200)           240800    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_6 (Dense)              (None, 150)               30150     
_________________________________________________________________
dropout_3 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 11299)             1706149   
Total params: 3,427,899
Trainable params: 3,427,899
Non-trainable params: 0
____________________________________________

In [179]:
model_v1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [180]:
history = model_v1.fit(
    X_train,
    Y_train,
    epochs = 300,
    batch_size = 547
)

Train on 40000 samples
Epoch 1/300
40000/40000 [==============================] - 7s 166us/sample - loss: 7.0790 - accuracy: 0.0427
Epoch 2/300
40000/40000 [==============================] - 4s 101us/sample - loss: 6.3214 - accuracy: 0.0442
Epoch 3/300
40000/40000 [==============================] - 4s 100us/sample - loss: 6.2927 - accuracy: 0.0446
Epoch 4/300
40000/40000 [==============================] - 4s 102us/sample - loss: 6.2379 - accuracy: 0.0439
Epoch 5/300
40000/40000 [==============================] - 4s 102us/sample - loss: 6.1331 - accuracy: 0.0474
Epoch 6/300
40000/40000 [==============================] - 4s 102us/sample - loss: 6.0062 - accuracy: 0.0612
Epoch 7/300
40000/40000 [==============================] - 4s 102us/sample - loss: 5.9059 - accuracy: 0.0702
Epoch 8/300
40000/40000 [==============================] - 4s 102us/sample - loss: 5.8325 - accuracy: 0.0771
Epoch 9/300
40000/40000 [==============================] - 4s 102us/sample - loss: 5.7729 - accuracy: 0.0

In [213]:
def continue_text(model, text_sequence, predicted_size):
    tokenized_text = tokenizer.texts_to_sequences([text_sequence])
    predicted_size = predicted_size + len(tokenized_text[0])

    while len(tokenized_text[0]) < predicted_size:
        padded_sentence = pad_sequences(
            tokenized_text[-(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED):], 
            maxlen=(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED)
        )
        
        op = model.predict(np.asarray(padded_sentence).reshape(1, -1))
        tokenized_text[0].append(op.argmax() + 1)
        
    return ' '.join(map(lambda x: dict(map(reversed, tokenizer.word_index.items()))[x], tokenized_text[0]))

In [214]:
continue_text(model_v1, "it might be increasingly difficult to remain quiet about a", 31)

'it might be increasingly difficult to remain quiet about a certain role a female author as if there is a day to how the dust is as likely to work closure or crave upon things from it this invitation could come'

In [200]:
X_test.iloc[3]

"it might be increasingly difficult to remain quiet about a particular matter you've been keeping to yourself or possibly feel strongly about you could be aware of the proverbial can of worms you'll open by instigating a particular conversation however you're only adding to your own stress levels by opting to say nothing if truth forms the basis of what you want to say then you have nothing to fear"

In [206]:
model_v1.save('./Documents/scripts/model_v1.2020.05.03.1.h5')

In [235]:
with open('./Documents/scripts/dictionary.json', 'w') as f:
    f.write(json.dumps(list(tokenizer.word_index.keys())))